<img src="https://github.com/vhertel/radar-based-flood-mapping/blob/main/resources/header.png?raw=1" width="1000"/>


# Radar-based Flood Mapping

<img src="https://github.com/vhertel/radar-based-flood-mapping/blob/main/resources/example.png?raw=1" width="1000"/>

***

The objective of this [Recommended Practice](https://un-spider.org/advisory-support/recommended-practices) is to determine the extent of flooded areas. The usage of Synthetic Aperture Radar (SAR) satellite imagery for flood extent mapping constitutes a viable solution with fast image processing, providing near real-time flood information to relief agencies for supporting humanitarian action. The high data reliability as well as the absence of geographical constraints, such as site accessibility, emphasize the technology’s potential in the field.

This Jupyter Notebook covers the full processing chain from data query and download up to the export of a final flood mask product by utilizing open access Sentinel-1 SAR data. The tool's workflow follows the UN-SPIDER Recommended Practice on [Radar-based Flood Mapping](https://un-spider.org/advisory-support/recommended-practices/recommended-practice-radar-based-flood-mapping) and is illustrated in the chart below. After entering user specifications, Sentinel-1  data can directly be downloaded from the <a href="https://scihub.copernicus.eu/">Copernicus Open Access Hub</a>. Subsequently, the data is processed and stored in a variety of output formats.

<div class="alert alert-danger" role="alert">
<b><i>Limitations</i></b><br />
Binder provides 2 GB of RAM which is sufficient for processing small areas. Thus, it is recommended to only select small AOIs and to restart the kernel after each processing cycle.  
Furthermore, the double bounce backscatter phenomenon causes difficulties in detecting flooded vegetation and floods in urban areas. If water and non-water are very unequally distributed in the image, the histogram might not have a clear local minimum, leading to incorrect results in the automatic binarization process.
</div> 

***

## User Input

<img src="https://github.com/vhertel/radar-based-flood-mapping/blob/main/resources/charts/chart1.png?raw=1" width="1000"/>

Please specify in the code cell below i) the polarisation to be processed and ii) whether data shall be downloaded from the <a href="https://scihub.copernicus.eu/">Copernicus Open Access Hub</a> with respective sensing period and login details.

In [ ]:
# polarisations to be processed
polarisations = 'VH'                              # 'VH', 'VV', 'both'

# download image from Copernicus Open Access Hub
downloadImage = True                              # True, False

## Initialization

This section loads relevant Python modules for the following analysis and initializes basic functionalities.

In [ ]:
# Click to run

#####################################################
###################### IMPORTS ######################
#####################################################

# MODULE                                      # DESCRIPTION
import sys
import matplotlib.pyplot as plt               # create visualizations
import numpy as np                            # scientific comupting
import json                                   # JSON encoder and decoder
import glob                                   # data access
import os                                     # data access
import ipywidgets                             # interactive UI controls
import time                                   # time assessment
import shutil                                 # file operations
import ipyleaflet                             # visualization
import geopandas                              # data analysis and manipulation
import snappy                                 # SNAP Python interface
import jpy                                    # Python-Java bridge
import skimage.filters                        # threshold calculation
import functools                              # higher-order functions and operations
from ipyfilechooser import FileChooser        # file chooser widget
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface to Open Access Hub
from datetime import date                     # dates, times and intervalls
from IPython.display import display, FileLink # visualization
from osgeo import ogr, gdal, osr              # data conversion
from zipfile import ZipFile                   # file management
    



####################################################
####################### CODE #######################
####################################################   
        
# get current working directory
directory = os.getcwd()

## Download Image

<img src="https://github.com/vhertel/radar-based-flood-mapping/blob/main/resources/charts/chart2.png?raw=1" width="1000"/>

This section allows interactive data access and download from the <a href="https://scihub.copernicus.eu/">Copernicus Open Access Hub</a>. The area of interest (AOI) can be selected and manipulated by using the drawing tool in the interactive map. Clicking the Search button below the map will load available images. If multiple AOIs are drawn, only the last one is considered. When hovering over a Sentinel-1 image, the tile index and ingestion date are shown. The table below summarizes information on all available tiles and allows the download. The Open Access Hub maintains an online archive of at least the latest year of products for immediate download. Access to previous products that are no longer available online will automatically trigger the retrieval from the long term archives. The actual download can be initiated by the user once the data are restored (within 24 hours).

In [ ]:
# Click to run

####################################################
############### FUNCTION DEFINITIONS ###############
####################################################

# search for and display available Sentinel-1 tiles
def queri(footprint):
    # print status
    print('Loading...\n', flush=True)
    # search Copernicus Open Access Hub for products with regard to input footprint and sensing period
    try:
        # connect to the API
        api = SentinelAPI(grid[0,0].value, grid[1,0].value, 'https://scihub.copernicus.eu/dhus')
        #print(date(download['period_start'][0], download['period_start'][1], download['period_start'][2]))
        #print(date(download['period_stop'][0], download['period_stop'][1], download['period_stop'][2]))
        products = api.query(footprint,
                             date = (grid[0,1].value, grid[1,1].value),
                             platformname = 'Sentinel-1',
                             producttype = 'GRD')
        print('Successfully connected to Copernicus Open Access Hub.\n', flush=True)
    except:
        raise ConnectionRefusedError
    # convert to GeoJSON for plot
    products_json = api.to_geojson(products)
    # raise warning that no image is available in given sensing period
    if not products_json['features']:
        raise FileNotFoundError
    # convert to dataframe for table visualization
    products_df = api.to_dataframe(products).sort_values('ingestiondate', ascending=[False])
    # adds index to dataframe
    indices = []
    for i in range (1, len(products_df.index)+1):
        indices.append('Tile %d' % i)
        products_json.features[i-1].properties['index'] = ' Tile %d' % i
    products_df.insert(0, 'index', indices, True) 
    # plot available Sentinel-1 tiles
    geo_json = ipyleaflet.GeoJSON(data = products_json,
                                  name = 'S1 tiles',
                                  style = {'color' : 'royalblue'},
                                  hover_style = {'fillOpacity' : 0.4})
    download_map.add_layer(geo_json)
    # call click_handler function when user clicks on Sentinel-1 tile
    tile_ID = geo_json.on_hover(hover_handler)
    # print table with download buttons
    download_grid = ipywidgets.GridspecLayout(len(products_df.index)+1, 5, height='250px')
    download_grid[0,0] = ipywidgets.HTML('<h4>Index</h4>')
    download_grid[0,1] = ipywidgets.HTML('<h4>Date</h4>')
    download_grid[0,2] = ipywidgets.HTML('<h4>Polarisation</h4>')
    download_grid[0,3] = ipywidgets.HTML('<h4>Size</h4>')
    for i in range(len(products_df.index)):
        download_grid[i+1,0] = ipywidgets.Label(products_df['index'][i])
        download_grid[i+1,1] = ipywidgets.Label(str(products_df['beginposition'][i]))
        download_grid[i+1,2] = ipywidgets.Label(products_df['polarisationmode'][i])
        download_grid[i+1,3] = ipywidgets.Label(products_df['size'][i])
        download_grid[i+1,4] = ipywidgets.Button(description = 'Download')
        download_grid[i+1,4].on_click(functools.partial(on_downloadButton_clicked, api=api, tile_id=products_df.values[i][-1]))
    display(download_grid)
    
# show product index and date on HTML element when hovering on Sentinel-1 tile 
def hover_handler(feature, **kwargs):
    # reset HTML widget and set new values
    html.value = '''
    Index: {}<br/>
    <small>Date: {}</small>
    '''.format(feature['properties']['index'], feature['properties']['beginposition'])

# search for available Sentinel-1 tiles with regard to AOI
def on_searchButton_clicked(b):
    # refocus of map to cover Sentinel-1 tiles
    download_map.zoom = 6.5
    # create WKT element with geographic coordinates of AOI
    coordinates = draw_control.last_draw['geometry']['coordinates'][0]
    lng1, lat1 = coordinates[0][0], coordinates[0][1]
    lng2, lat2 = coordinates[1][0], coordinates[1][1]
    lng3, lat3 = coordinates[2][0], coordinates[2][1]
    lng4, lat4 = coordinates[3][0], coordinates[3][1]
    footprint = 'POLYGON((%s %s, %s %s, %s %s, %s %s, %s %s))' % (lng1, lat1, lng2, lat2, lng3, lat3, lng4, lat4, lng1, lat1)
    # call function to search for available Sentinel-1 tiles
    try:
        queri(footprint)
    except ConnectionRefusedError:
        print('Login data not valid. Please change username and/or password.\n')
    except FileNotFoundError:
        print('No Sentinel-1 images available. Please change sensing period in user input section.\n')
        
# download chosen Sentinel-1 tile in subfolder 'input'
def on_downloadButton_clicked(b, api, tile_id):
    # get product information
    product_info = api.get_product_odata(tile_id)
    # check whether product is available
    if product_info['Online']:
        # check if input folder exists, if not create input folder
        input_path = os.path.join(directory, 'input')
        if not os.path.isdir(input_path):
            os.mkdir(input_path)
        # change into 'input' subfolder for storing product
        os.chdir(input_path)
        # status update
        print('\nProduct %s is online. Starting download.' % tile_id, flush=True)
        # download product
        api.download(tile_id)
        # change back to previous working directory
        os.chdir(directory)
    # error message when product is not available
    else:
        print('\nProduct %s is not online. Must be requested manually.\n' % tile_id, flush=True)




####################################################
####################### CODE #######################
####################################################

# check user input whether image download is requested
if downloadImage:
    # create map with search functionality
    download_map = ipyleaflet.Map(zoom = 1.4)
    download_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    display(download_map)
    
    # create HTML widget to display product index and date of selected Sentinel-1 tile
    html = ipywidgets.HTML('')
    control = ipyleaflet.WidgetControl(widget=html, position='topright')
    download_map.add_control(control)
    
    # create draw control function
    draw_control = ipyleaflet.DrawControl(circlemarker={},
                                          polyline={},
                                          polygon={'shapeOptions':{'color':'green'}},
                                          rectangle={'shapeOptions':{'color':'green'}})
    # adds draw control function to map
    download_map.add_control(draw_control)
        
    # print table with download buttons
    grid = ipywidgets.GridspecLayout(2, 2)#, height='250px')
    grid[0,0] = ipywidgets.Text(description='Username:')
    grid[0,1] = ipywidgets.DatePicker(description='Start Date')
    grid[1,0] = ipywidgets.Password(description='Password:')
    grid[1,1] = ipywidgets.DatePicker(description='Stop Date')      
    display(grid)

    # create search button and call function to search for available Sentinel-1 tiles when clicked
    searchButton = ipywidgets.Button(description = 'Search')
    searchButton.on_click(on_searchButton_clicked)
    display(searchButton)

## Processing & Data Export

<img src="https://github.com/vhertel/radar-based-flood-mapping/blob/main/resources/charts/chart34.png?raw=1" width="1000"/>

If more than one Sentinel-1 image has been downloaded, the user can select which one is to be used for the processing. An interactive map allows drawing the area of interest for generating a subset. Subsequently, the following processing steps are performed:

1. ***Apply Orbit File***: The orbit file provides accurate satellite position and velocity information. Based on this information, the orbit state vectors in the abstract metadata of the product are updated. The precise orbit files are available days-to-weeks after the generation of the product. Since this is an optional processing step, the tool will continue the workflow in case the orbit file is not yet available to allow rapid mapping applications.


2. ***Thermal Noise Removal***: Thermal noise correction is applied to Sentinel-1 Level-1 GRD products which have not already been corrected.


3. ***Radiometric Calibration***: The objective of SAR calibration is to provide imagery in which the pixel values can be directly related to the radar backscatter of the scene. Though uncalibrated SAR imagery is sufficient for qualitative use, calibrated SAR images are essential to the quantitative use of SAR data.


4. ***Speckle Filtering***: SAR images have inherent texturing called speckles which degrade the quality of the image and make interpretation of features more difficult. Speckles are caused by random constructive and destructive interference of the de-phased but coherent return waves scattered by the elementary scatter within each resolution cell. Speckle noise reduction can be applied either by spatial filtering or multilook processing. A Lee filter with an X, Y size of 5, 5 is used in this step.


5. ***Terrain Correction***: Due to topographical variations of a scene and the tilt of the satellite sensor, distances can be distorted in the SAR images. Data which is not directly directed towards the sensor’s Nadir location will have some distortion. Therefore, terrain corrections are intended to compensate for these distortions to allow a realistic geometric representation in the image.


6. ***Binarization***: In order to obtain a binary flood mask, the histogram is analyzed to separate water from non-water pixels. Due to the side-looking geometry of SAR sensors and the comparably smooth surface of water, only a very small proportion of backscatter is reflected back to the sensor leading to comparably low pixel values in the histogram. The threshold used for separation is automatically calculated using <a href="https://scikit-image.org/">scikit-image</a> implementations and a combined use of the <a href="https://doi.org/10.1111/j.1749-6632.1965.tb11715.x">minimum method</a> and <a href="https://www.semanticscholar.org/paper/A-threshold-selection-method-from-gray-level-Otsu/1d4816c612e38dac86f2149af667a5581686cdef?p2df">Otsu's method</a>. The <a href="http://due.esrin.esa.int/page_globcover.php">GlobCover</a> layer of the European Space Agency is used to mask out permanent water bodies.


7. ***Speckle Filtering***: A Median filter with an X, Y size of 7, 7 is used in this step.


The processed flood mask is exported as GeoTIFF, SHP, KML, and GeoJSON and can directly be downloaded. An interactive map shows the flood mask.

In [ ]:
# Click to run

####################################################
############### FUNCTION DEFINITIONS ###############
####################################################

# create S1 product
def getScene(path):
    # set correct path of input file and create S1 product
    if len(files) is 1:
        file_path = path
    else:
        file_path = path.selected
    S1_source = snappy.ProductIO.readProduct(file_path)

    # read geographic coordinates from Sentinel-1 image meta data
    meta_data = S1_source.getMetadataRoot().getElement('Abstracted_Metadata')
    # refines center of map according to Sentinel-1 image
    center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))
    locations = [[{'lat' : meta_data.getAttributeDouble('first_near_lat'), 'lng' : meta_data.getAttributeDouble('first_near_long')},
                  {'lat' : meta_data.getAttributeDouble('last_near_lat'),  'lng' : meta_data.getAttributeDouble('last_near_long')},
                  {'lat' : meta_data.getAttributeDouble('last_far_lat'),   'lng' : meta_data.getAttributeDouble('last_far_long')},
                  {'lat' : meta_data.getAttributeDouble('first_far_lat'),  'lng' : meta_data.getAttributeDouble('first_far_long')}]]

    # creates interactive map
    basic_map = ipyleaflet.Map(center = center, zoom = 7.5)
    # defines fixed polygon illustrating Sentinel-1 image
    polygon_fix = ipyleaflet.Polygon(locations = locations, color='royalblue')
    basic_map.add_layer(polygon_fix)
    # displays map
    basic_map.add_control(ipyleaflet.FullScreenControl())
    basic_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    display(basic_map)
    
    # editable polygon determining AOI
    polygon_flex = ipyleaflet.Polygon(locations = locations,
                                      color = 'green',
                                      fill_color = 'green',
                                      transform = True)
    basic_map.add_layer(polygon_flex)
    # create process button and call function to start ptocessing when clicked
    processButton = ipywidgets.Button(description = 'Start Processing')
    processButton.on_click(functools.partial(on_processButton_clicked,
                                             S1_source = S1_source,
                                             polygon_flex = polygon_flex))
    display(processButton)
    
# calculate and return threshold of 'Band'-type input
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def getThreshold(S1_band):
    # read band
    w = S1_band.getRasterWidth()
    h = S1_band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    S1_band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h * w

    # calculate threshold using Otsu method
    threshold_otsu = skimage.filters.threshold_otsu(band_data)
    # calculate threshold using minimum method
    threshold_minimum = skimage.filters.threshold_minimum(band_data)
    # get number of pixels for both thresholds
    numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
    numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # if number of pixels at minimum threshold is less than 1% of number of pixels at Otsu threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        # adjust band data according
        if threshold_otsu < threshold_minimum:
            band_data = band_data[band_data < threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        else:
            band_data = band_data[band_data > threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
    
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # select final threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        threshold = threshold_otsu
    else:
        threshold = threshold_minimum

    return threshold

# calculate binary mask of 'Product'-type intput with respect expression in string array
def binarization(S1_product, expressions):

    BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
    targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(expressions))

    # loop through bands
    for i in range(len(expressions)):
        targetBand = BandDescriptor()
        targetBand.name = '%s' % S1_product.getBandNames()[i]
        targetBand.type = 'float32'
        targetBand.expression = expressions[i]
        targetBands[i] = targetBand
    
    parameters = snappy.HashMap()
    parameters.put('targetBands', targetBands)    
    mask = snappy.GPF.createProduct('BandMaths', parameters, S1_product)

    return mask

# start processing button 
def on_processButton_clicked(b, S1_source, polygon_flex):
    # get coordinates and store in WKT format variable
    lng1, lat1 = polygon_flex.locations[0][0]['lng'], polygon_flex.locations[0][0]['lat']
    lng2, lat2 = polygon_flex.locations[0][1]['lng'], polygon_flex.locations[0][1]['lat']
    lng3, lat3 = polygon_flex.locations[0][2]['lng'], polygon_flex.locations[0][2]['lat']
    lng4, lat4 = polygon_flex.locations[0][3]['lng'], polygon_flex.locations[0][3]['lat']
    footprint = 'POLYGON((%s %s, %s %s, %s %s, %s %s, %s %s))' % (lng1, lat1, lng2, lat2, lng3, lat3, lng4, lat4, lng1, lat1)
    # run processing
    processing(S1_source, footprint)

# processing steps
def processing(S1_source, footprint):
    
    # Subset operator
    parameters = snappy.HashMap()
    parameters.put('copyMetadata', True)
    geom = snappy.WKTReader().read(footprint)
    parameters.put('geoRegion', geom)
    parameters.put('sourceBands', sourceBands)
    S1_crop = snappy.GPF.createProduct('Subset', parameters, S1_source)
    # status update
    print('\nSubset successfully generated.\n', flush=True)
    
    # Apply-Orbit-File operator
    print('1. Apply Orbit File:          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    # continue with calculation in case no orbit file is available yet
    parameters.put('continueOnFail', True)
    S1_Orb = snappy.GPF.createProduct('Apply-Orbit-File', parameters, S1_crop)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # ThermalNoiseRemoval operator
    print('2. Thermal Noise Removal:     ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('removeThermalNoise', True)
    S1_Thm = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, S1_Orb)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Calibration operator
    print('3. Radiometric Calibration:   ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('outputSigmaBand', True)
    S1_Cal = snappy.GPF.createProduct('Calibration', parameters, S1_Thm)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Speckle-Filter operator
    print('4. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    S1_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_Cal)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Conversion from linear to db operator
    S1_Spk_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1_Spk)

    # Terrain-Correction operator
    print('5. Terrain Correction:        ', end='', flush=True)
    parameters = snappy.HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
    parameters.put('pixelSpacingInMeter', 10.0)
    parameters.put('nodataValueAtSea', False)
    parameters.put('saveSelectedSourceBand', True)
    S1_TC = snappy.GPF.createProduct('Terrain-Correction', parameters, S1_Spk_db)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Binarization
    print('6. Binarization:              ', end='', flush=True)
    start_time = time.time()
    # add GlobCover band
    parameters = snappy.HashMap()
    parameters.put('landCoverNames', 'GlobCover')
    GlobCover = snappy.GPF.createProduct('AddLandCover', parameters, S1_TC)
    # empty string array for binarization band maths expression(s)
    expressions = ['' for i in range(S1_TC.getNumBands())]
    # empty array for threshold(s)
    thresholds = np.zeros(S1_TC.getNumBands())
    # loop through bands
    for i in range(S1_TC.getNumBands()):
        # calculate threshold of band and store in float array
        # use S1_Spk_db product for performance reasons. S1_TC causes 0-values
        # which distort histogram and thus threshold result
        thresholds[i] = getThreshold(S1_Spk_db.getBandAt(i))
        # formulate expression according to threshold and store in string array
        expressions[i] = 'if (%s < %s && land_cover_GlobCover != 210) then 1 else NaN' % (S1_TC.getBandNames()[i], thresholds[i])
    # do binarization
    S1_floodMask = binarization(GlobCover, expressions)
    print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

    # Speckle-Filter operator
    print('7. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Median')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    # define flood mask as global for later access
    S1_floodMask_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_floodMask)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    export(S1_floodMask_Spk)
    
# prepare output files for download
def export(S1_floodMask_Spk):
    print('\n___________________________________________________\n', flush=True)
    print('\nPreparing output data...\n', flush=True)
    # check if output folders exists, if not create folders
    output_path = os.path.join(directory, 'output')
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    GeoTIFF_path = os.path.join(output_path, 'GeoTIFF')
    if not os.path.isdir(GeoTIFF_path):
        os.mkdir(GeoTIFF_path)
    SHP_path = os.path.join(output_path, 'SHP')
    if not os.path.isdir(SHP_path):
        os.mkdir(SHP_path)
    KML_path = os.path.join(output_path, 'KML')
    if not os.path.isdir(KML_path):
        os.mkdir(KML_path)
    GeoJSON_path = os.path.join(output_path, 'GeoJSON')
    if not os.path.isdir(GeoJSON_path):
        os.mkdir(GeoJSON_path)
    # get file name if file chooser was used
    if len(files) is 1:
        input_name = files[0]
    else:
        input_name = fc.selected_filename

    # write output file as GeoTIFF
    snappy.ProductIO.writeProduct(S1_floodMask_Spk, '%s/%s_%s' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions), 'GeoTIFF')

    # convert GeoTIFF to SHP
    # allow GDAL to throw Python exceptions
    gdal.UseExceptions()
    open_image = gdal.Open('%s/%s_%s.tif' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions))
    srs = osr.SpatialReference()
    srs.ImportFromWkt(open_image.GetProjectionRef())
    shp_driver = ogr.GetDriverByName('ESRI Shapefile')
    # empty string array for bands in GeoTIFF
    output_shp = ['' for i in range(open_image.RasterCount)]
    if open_image.RasterCount == 1:
        output_shp[0] = '%s/%s_processed_%s' % (SHP_path, os.path.splitext(input_name)[0], polarisations)
    else:
        VH_SHP_path = os.path.join(SHP_path, 'VH')
        if not os.path.isdir(VH_SHP_path):
            os.mkdir(VH_SHP_path)
        VV_SHP_path = os.path.join(SHP_path, 'VV')
        if not os.path.isdir(VV_SHP_path):
            os.mkdir(VV_SHP_path)
        output_shp[0] = '%s/%s_processed_VH' % (VH_SHP_path, os.path.splitext(input_name)[0])
        output_shp[1] = '%s/%s_processed_VV' % (VV_SHP_path, os.path.splitext(input_name)[0])
    # loops through bands in GeoTIFF
    for i in range(open_image.RasterCount):
        input_band = open_image.GetRasterBand(i+1)
        output_shapefile = shp_driver.CreateDataSource(output_shp[i] + '.shp')
        new_shapefile = output_shapefile.CreateLayer(output_shp[i], srs=srs)
        new_shapefile.CreateField(ogr.FieldDefn('DN', ogr.OFTInteger))
        gdal.Polygonize(input_band, input_band.GetMaskBand(), new_shapefile, 0, [], callback=None)
        # filters attributes with values other than 1 (sould be NaN or respective value)
        new_shapefile.SetAttributeFilter('DN != 1')
        for feat in new_shapefile:
            new_shapefile.DeleteFeature(feat.GetFID())
        new_shapefile.SyncToDisk()

    # convert SHP to KML
    if open_image.RasterCount == 1:
        shp_file = gdal.OpenEx('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], polarisations))
        ds = gdal.VectorTranslate('%s/%s_processed_%s.kml' % (KML_path, os.path.splitext(input_name)[0], polarisations), shp_file, format='KML')
        del ds
    else:
        shp_file_VH = gdal.OpenEx('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
        ds_VH = gdal.VectorTranslate('%s/%s_processed_VH.kml' % (KML_path, os.path.splitext(input_name)[0]), shp_file_VH, format='KML')
        del ds_VH
        shp_file_VV = gdal.OpenEx('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
        ds_VV = gdal.VectorTranslate('%s/%s_processed_VV.kml' % (KML_path, os.path.splitext(input_name)[0]), shp_file_VV, format='KML')
        del ds_VV

    # convert SHP to GeoJSON
    if open_image.RasterCount == 1:
        shp_file = geopandas.read_file('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], polarisations))
        shp_file.to_file('%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], polarisations), driver='GeoJSON')
    else:
        shp_file_VH = geopandas.read_file('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
        shp_file_VH.to_file('%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')    
        shp_file_VV = geopandas.read_file('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
        shp_file_VV.to_file('%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')

    # create zip files for data download
    shutil.make_archive('%s/GeoTIFF__%s' % (output_path, os.path.splitext(input_name)[0]), 'zip', GeoTIFF_path)
    shutil.make_archive('%s/SHP__%s' % (output_path, os.path.splitext(input_name)[0]), 'zip', SHP_path)
    shutil.make_archive('%s/KML__%s' % (output_path, os.path.splitext(input_name)[0]), 'zip', KML_path)
    shutil.make_archive('%s/GeoJSON__%s' % (output_path, os.path.splitext(input_name)[0]), 'zip', GeoJSON_path)
    display(FileLink('output/GeoTIFF__%s.zip' % os.path.splitext(input_name)[0],    result_html_prefix='Download GeoTIFF:   '))
    display(FileLink('output/SHP__%s.zip' % os.path.splitext(input_name)[0],     result_html_prefix='Download SHP:       '))
    display(FileLink('output/KML__%s.zip' % os.path.splitext(input_name)[0],     result_html_prefix='Download KML:       '))
    display(FileLink('output/GeoJSON__%s.zip' % os.path.splitext(input_name)[0], result_html_prefix='Download GeoJSON:   '))
    
    # plot results
    results_map = ipyleaflet.Map(zoom=9, basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik)    
    if open_image.RasterCount == 1:
        file = '%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], polarisations)
        with open(file, 'r') as f:
            data_json = json.load(f) 
        mask = ipyleaflet.GeoJSON(data = data_json, name = 'Flood Mask', style = {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'})
        results_map.add_layer(mask)
        results_map.center = (mask.data['features'][0]['geometry']['coordinates'][0][0][1],
                              mask.data['features'][0]['geometry']['coordinates'][0][0][0])
    else:
        file_VV = '%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
        with open(file_VV, 'r') as f_VV:
            data_json_VV = json.load(f_VV)
        mask_VV = ipyleaflet.GeoJSON(data = data_json_VV, name = 'Flood Mask: VV', style = {'color':'red', 'opacity':'1', 'fillColor':'red', 'fillOpacity':'1', 'weight':'0.8'})
        results_map.add_layer(mask_VV)
        results_map.center = (mask_VV.data['features'][0]['geometry']['coordinates'][0][0][1],
                              mask_VV.data['features'][0]['geometry']['coordinates'][0][0][0])  
        file_VH = '%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
        with open(file_VH, 'r') as f_VH:
            data_json_VH = json.load(f_VH)
        mask_VH = ipyleaflet.GeoJSON(data = data_json_VH, name = 'Flood Mask: VH', style = {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'})
        results_map.add_layer(mask_VH)
    results_map.add_control(ipyleaflet.FullScreenControl())
    results_map.add_control(ipyleaflet.LayersControl(position='topright'))
    results_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    display(results_map)
    
    # remove unzipped directories and data
    shutil.rmtree(GeoTIFF_path)
    shutil.rmtree(SHP_path)
    shutil.rmtree(KML_path)
    shutil.rmtree(GeoJSON_path)




####################################################
####################### CODE #######################
####################################################

# filter required polarisation(s) and set output file name accordingly
if polarisations == 'both':
    sourceBands = 'Amplitude_VH,Intensity_VH,Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VHVV'
elif polarisations == 'VH':
    sourceBands = 'Amplitude_VH,Intensity_VH'
    output_extensions   = 'processed_VH'
elif polarisations == 'VV':
    sourceBands = 'Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VV'

# path of Sentinel-1 .zip input file
input_path = os.path.join(directory, 'input')
# empty string array to store Sentinel-1 files in 'input' subfolder
files = []
# add files to list
for file in glob.glob1(input_path, '*.zip'):
    files.append(file)
# select input file and start processing if there is only one available Sentinel-1 file
if len(files) == 1:
    input_name = files[0]
    print('Selected:  %s\n' % input_name, flush=True)
    # apply subset according to JSON data
    getScene('%s/%s' % (input_path, input_name))
# open dialogue to select input file if more or less than one is available
else:
    print('More or less than one Sentinel-1 files have been found. Please select.', flush=True)
    fc = FileChooser(input_path)
    fc.filter_pattern = '*.zip'
    fc.register_callback(getScene)
    display(fc)